<a href="https://colab.research.google.com/github/tylerwang26/nodejs.org/blob/main/Copy_of_agno_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/0x-yuan/clintrial-nlp/blob/main/agno_baseline.ipynb)

# 協作智能思考框架基線 - 臨床試驗 NLP

## 概述

本notebook展示如何使用簡單的協作思考模式進行臨床試驗自然語言推理(NLI)。我們模擬"團隊協作"概念，讓多個專業角色協同分析，最終達成共識。

## 📚 學習目標
- 理解協作思考的重要性
- 建立專業角色協作模式
- 實作團隊共識決策過程
- 評估系統效能

### 🤝 協作思考架構
我們實作4個協作角色：
1. **臨床研究主管**: 總體協調和醫學專業指導
2. **資料科學家**: 統計分析和數據驗證
3. **品質保證員**: 邏輯檢查和品質控制
4. **決策委員會**: 整合所有意見並達成共識

> 💡 **核心概念**: 每個角色都有獨特的專業視角，通過協作討論來提高決策品質。

In [ ]:
# 🔧 環境設置 - 一鍵安裝所需套件
!pip install -q google-generativeai python-dotenv pandas tqdm gdown

print("✅ 所有套件安裝完成！")

✅ 所有套件安裝完成！


In [ ]:
# 📥 從 Google Drive 下載訓練資料
import os
import gdown
import zipfile
import shutil

# Google Drive zip 檔案 ID
file_id = "15GA5XI39DDxQ5QkIZXsFbApx1yEvCpcR"
zip_url = f"https://drive.google.com/uc?id={file_id}"
zip_filename = "clinicaltrial-nlp.zip"

if not os.path.exists("training_data"):
    print("📥 從 Google Drive 下載 clinicaltrial-nlp.zip...")
    try:
        gdown.download(zip_url, zip_filename, quiet=False)

        print("📦 正在解壓縮檔案...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(".")

        if os.path.exists("clintrial-nlp/training_data"):
            shutil.move("clintrial-nlp/training_data", "training_data")
            if os.path.exists("clintrial-nlp"):
                shutil.rmtree("clintrial-nlp")

        os.remove(zip_filename)
        print("✅ 訓練資料下載並解壓縮完成！")

    except Exception as e:
        print(f"❌ 下載失敗: {e}")
        print("請手動下載: https://drive.google.com/file/d/15GA5XI39DDxQ5QkIZXsFbApx1yEvCpcR/view")
else:
    print("✅ 訓練資料已存在，跳過下載")

# 檢查下載的資料
if os.path.exists("training_data/CT json"):
    ct_files = len([f for f in os.listdir("training_data/CT json") if f.endswith('.json')])
    print(f"📄 找到 {ct_files} 個臨床試驗JSON檔案")

📥 從 Google Drive 下載 clinicaltrial-nlp.zip...


Downloading...
From: https://drive.google.com/uc?id=15GA5XI39DDxQ5QkIZXsFbApx1yEvCpcR
To: /content/clinicaltrial-nlp.zip
100%|██████████| 5.47M/5.47M [00:00<00:00, 201MB/s]


📦 正在解壓縮檔案...
✅ 訓練資料下載並解壓縮完成！
📄 找到 999 個臨床試驗JSON檔案


In [ ]:
# 🧪 準備測試資料集
import json

def create_test_data_if_needed():
    if not os.path.exists("test.json"):
        try:
            with open("training_data/train.json", "r", encoding="utf-8") as f:
                train_data = json.load(f)
            test_data = dict(list(train_data.items())[:100])
            with open("test.json", "w", encoding="utf-8") as f:
                json.dump(test_data, f, indent=2, ensure_ascii=False)
            print(f"✅ 已創建測試資料集，包含 {len(test_data)} 個樣本")
        except Exception as e:
            print(f"❌ 創建測試資料失敗: {e}")
    else:
        print("✅ test.json 已存在")

create_test_data_if_needed()

✅ 已創建測試資料集，包含 100 個樣本


In [ ]:
# 載入環境變數和必要函式庫
from dotenv import load_dotenv
import os
import json
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai
import time
import warnings
warnings.filterwarnings('ignore')

load_dotenv()
print("✅ 環境變數載入完成")

✅ 環境變數載入完成


## 模型配置

配置Google Gemini模型進行推理：

In [ ]:
# 配置 Google Gemini 模型
from google.colab import userdata
api_key = os.getenv("GEMINI_API_KEY") or userdata.get("GOOGLE_API_KEY")

if not api_key:
    print("⚠️ 請設定 GOOGLE_API_KEY 環境變數")
    print("可以在 Colab 左側面板的 'Secrets' 中設定")
    raise ValueError("缺少 API 金鑰")
else:
    print(f"✅ 找到 API 金鑰: {api_key[:8]}...{api_key[-4:]}")

genai.configure(api_key=api_key)

# 測試 API 連接
try:
    test_model = genai.GenerativeModel("gemini-2.5-flash")
    test_response = test_model.generate_content("Hello, respond with 'API test successful'")
    print(f"✅ API 連接測試成功: {test_response.text[:50]}...")
except Exception as e:
    print(f"❌ API 連接測試失敗: {e}")
    raise

# 創建 Gemini 模型實例
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    generation_config=genai.types.GenerationConfig(
        temperature=0.1,
        max_output_tokens=4096,
        top_p=1,
        top_k=1
    )
)

print(f"✅ Google Gemini模型配置完成")

✅ 找到 API 金鑰: AIzaSyDr...-V-4
✅ API 連接測試成功: API test successful...
✅ Google Gemini模型配置完成


## 資料工具函式

建立用於載入和處理臨床試驗資料的工具函式：

In [ ]:
def load_clinical_trial(trial_id: str) -> dict:
    """載入臨床試驗資料"""
    try:
        file_path = os.path.join("training_data", "CT json", f"{trial_id}.json")
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        return {"error": f"找不到臨床試驗 {trial_id}"}
    except Exception as e:
        return {"error": f"載入 {trial_id} 時發生錯誤: {str(e)}"}

def create_collaborative_context(trial_data: dict, focus_section: str = None) -> str:
    """創建協作討論的上下文"""
    if "error" in trial_data:
        return f"錯誤: {trial_data['error']}"

    sections = {
        "Eligibility": trial_data.get("Eligibility", []),
        "Intervention": trial_data.get("Intervention", []),
        "Results": trial_data.get("Results", []),
        "Adverse Events": trial_data.get("Adverse_Events", [])
    }

    context = [f"試驗編號: {trial_data.get('Clinical Trial ID', 'Unknown')}"]
    context.append("\n=== 試驗資料摘要 ===")

    if focus_section and focus_section in sections:
        section_data = sections[focus_section]
        context.append(f"\n{focus_section} 重點資訊:")
        if isinstance(section_data, list):
            for i, item in enumerate(section_data[:5]):
                context.append(f"  {i+1}. {item}")
            if len(section_data) > 5:
                context.append(f"  ... (另有 {len(section_data)-5} 項資訊)")
        else:
            context.append(f"  {section_data}")
    else:
        for section_name, section_data in sections.items():
            if section_data:
                context.append(f"\n{section_name}:")
                if isinstance(section_data, list):
                    for item in section_data[:2]:
                        context.append(f"  • {item}")
                    if len(section_data) > 2:
                        context.append(f"  • ... (共 {len(section_data)} 項)")
                else:
                    context.append(f"  • {section_data}")

    return "\n".join(context)

# 測試工具函式
sample_trial = load_clinical_trial("NCT00066573")
print(f"✅ 資料工具函式準備就緒。範例試驗: {sample_trial.get('Clinical Trial ID', '錯誤')}")

✅ 資料工具函式準備就緒。範例試驗: NCT00066573


## 協作角色定義

定義四個協作角色，每個角色都有特定的專業職責：

In [ ]:
def clinical_director_analysis(statement: str, trial_context: str) -> str:
    """臨床研究主管分析"""
    prompt = f"""你是臨床研究主管，負責從整體醫學角度協調分析。你具有豐富的臨床試驗經驗。

需要分析的陳述: "{statement}"

試驗資料:
{trial_context}

作為臨床研究主管，請提供你的專業分析：
1. 從醫學專業角度評估陳述的合理性
2. 考慮臨床試驗的整體背景和設計
3. 評估陳述與臨床實務的相符程度
4. 提供初步的醫學判斷

請提供簡潔的分析，最後以「主管意見: [支持/質疑/需要更多資訊]」結尾。"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"臨床主管分析錯誤: {str(e)}"

def data_scientist_analysis(statement: str, trial_context: str, director_input: str) -> str:
    """資料科學家分析"""
    prompt = f"""你是資料科學家，專精統計分析和數據驗證。你將協助臨床研究主管進行技術分析。

需要分析的陳述: "{statement}"

試驗資料:
{trial_context}

臨床主管的意見:
{director_input}

作為資料科學家，請協作分析：
1. 驗證陳述中涉及的數值和統計資訊
2. 檢查計算的準確性和統計方法
3. 評估數據解釋是否恰當
4. 與主管的醫學觀點進行技術對照

請提供技術分析，最後以「數據意見: [驗證通過/發現問題/數據不足]」結尾。"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"資料科學家分析錯誤: {str(e)}"

def quality_assurance_analysis(statement: str, director_input: str, data_input: str) -> str:
    """品質保證員分析"""
    prompt = f"""你是品質保證員，負責邏輯檢查和品質控制。你將整合團隊的不同觀點。

需要分析的陳述: "{statement}"

臨床主管的意見:
{director_input}

資料科學家的意見:
{data_input}

作為品質保證員，請協作分析：
1. 檢查陳述的邏輯一致性
2. 驗證推理過程是否合理
3. 評估團隊意見之間的一致性
4. 識別可能的品質風險或矛盾

請提供品質檢查結果，最後以「品質意見: [通過檢查/發現風險/需要澄清]」結尾。"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"品質保證分析錯誤: {str(e)}"

def decision_committee_consensus(statement: str, director_input: str, data_input: str, qa_input: str) -> str:
    """決策委員會共識"""
    prompt = f"""你是決策委員會主席，負責整合所有團隊成員的專業意見，達成最終共識。

需要決策的陳述: "{statement}"

臨床研究主管意見:
{director_input}

資料科學家意見:
{data_input}

品質保證員意見:
{qa_input}

作為委員會主席，請協調所有意見並做出最終決策：
1. 綜合評估各專業角度的意見
2. 權衡不同觀點的重要性
3. 識別關鍵的決策因素
4. 達成團隊共識

任務: 判斷陳述是「蘊含」(Entailment)還是「矛盾」(Contradiction)
- 蘊含: 陳述被試驗證據支持
- 矛盾: 陳述被試驗證據反駁

請提供簡要的團隊共識理由，然後以「委員會決議: [Entailment/Contradiction]」結尾。"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"委員會決議錯誤: {str(e)}"

print("✅ 四個協作角色定義完成")

✅ 四個協作角色定義完成


## 協作智能分析管道

創建協調所有角色的團隊協作分析管道：

In [ ]:
def collaborative_intelligence_pipeline(statement: str, primary_id: str, secondary_id: str = None,
                                       section_id: str = None, verbose: bool = False) -> str:
    """運行完整的協作智能分析管道"""

    try:
        # 準備協作上下文
        primary_data = load_clinical_trial(primary_id)
        trial_context = create_collaborative_context(primary_data, section_id)

        if secondary_id:
            secondary_data = load_clinical_trial(secondary_id)
            secondary_context = create_collaborative_context(secondary_data, section_id)
            trial_context += f"\n\n=== 次要試驗資料 ===\n{secondary_context}"

        if verbose:
            print(f"📄 協作分析陳述: {statement[:100]}...")
            print(f"🏥 主要試驗: {primary_id}")
            if secondary_id:
                print(f"🏥 次要試驗: {secondary_id}")

        # 第一輪: 臨床研究主管分析
        director_analysis = clinical_director_analysis(statement, trial_context)
        if verbose:
            print("👔 臨床研究主管: 完成初步分析")

        # 第二輪: 資料科學家協作分析
        data_analysis = data_scientist_analysis(statement, trial_context, director_analysis)
        if verbose:
            print("📊 資料科學家: 完成技術驗證")

        # 第三輪: 品質保證員整合分析
        qa_analysis = quality_assurance_analysis(statement, director_analysis, data_analysis)
        if verbose:
            print("🔍 品質保證員: 完成品質檢查")

        # 第四輪: 決策委員會達成共識
        consensus_decision = decision_committee_consensus(
            statement, director_analysis, data_analysis, qa_analysis
        )

        # 提取最終決策
        if "委員會決議: Entailment" in consensus_decision:
            decision = "Entailment"
        elif "委員會決議: Contradiction" in consensus_decision:
            decision = "Contradiction"
        else:
            # 備用解析
            if "entailment" in consensus_decision.lower() and "contradiction" not in consensus_decision.lower():
                decision = "Entailment"
            else:
                decision = "Contradiction"

        if verbose:
            print(f"🤝 決策委員會: 達成共識 - {decision}")
            print("-" * 50)

        return decision

    except Exception as e:
        if verbose:
            print(f"❌ 協作管道錯誤: {e}")
        return "Contradiction"  # 保守的備用方案

print("✅ 協作智能分析管道準備就緒")

✅ 協作智能分析管道準備就緒


## 測試範例

測試我們的協作智能系統：

In [ ]:
# 測試範例
test_statement = "there is a 13.2% difference between the results from the two the primary trial cohorts"
test_primary_id = "NCT00066573"

print(f"測試協作智能系統:")
print(f"陳述: '{test_statement}'")
print(f"主要試驗: {test_primary_id}")
print("\n" + "="*80)

# 執行分析
start_time = time.time()
result = collaborative_intelligence_pipeline(
    statement=test_statement,
    primary_id=test_primary_id,
    section_id="Results",
    verbose=True
)
end_time = time.time()

print(f"\n🎯 協作智能結果: {result}")
print(f"⏱️ 執行時間: {end_time - start_time:.2f} 秒")
print("="*80)

測試協作智能系統:
陳述: 'there is a 13.2% difference between the results from the two the primary trial cohorts'
主要試驗: NCT00066573

📄 協作分析陳述: there is a 13.2% difference between the results from the two the primary trial cohorts...
🏥 主要試驗: NCT00066573
👔 臨床研究主管: 完成初步分析
📊 資料科學家: 完成技術驗證
🔍 品質保證員: 完成品質檢查
🤝 決策委員會: 達成共識 - Contradiction
--------------------------------------------------

🎯 協作智能結果: Contradiction
⏱️ 執行時間: 71.24 秒


## 在訓練資料上評估

在訓練資料樣本上評估我們的系統：

In [ ]:
# 載入訓練資料
with open("training_data/train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)
print(f"載入 {len(train_data)} 個訓練範例")

# 在樣本上評估
sample_size = 12
examples = list(train_data.items())[:sample_size]

print(f"\n在 {len(examples)} 個範例上評估協作智能系統...")

results = []
correct = 0
total_time = 0

for i, (uuid, example) in enumerate(tqdm(examples, desc="協作智能處理")):
    try:
        statement = example.get("Statement")
        primary_id = example.get("Primary_id")
        secondary_id = example.get("Secondary_id")
        section_id = example.get("Section_id")
        expected = example.get("Label")

        if not statement or not primary_id:
            results.append({
                "uuid": uuid,
                "expected": expected,
                "predicted": "SKIPPED",
                "correct": False,
                "time": 0
            })
            continue

        # 獲取預測
        start_time = time.time()
        predicted = collaborative_intelligence_pipeline(
            statement=statement,
            primary_id=primary_id,
            secondary_id=secondary_id,
            section_id=section_id,
            verbose=False
        )
        end_time = time.time()

        execution_time = end_time - start_time
        total_time += execution_time

        # 檢查正確性
        is_correct = (predicted.strip() == expected.strip())
        if is_correct:
            correct += 1

        results.append({
            "uuid": uuid,
            "statement": statement[:80] + "..." if len(statement) > 80 else statement,
            "expected": expected,
            "predicted": predicted,
            "correct": is_correct,
            "time": execution_time
        })

        status = "✅" if is_correct else "❌"
        print(f"範例 {i+1:2d}: {expected:12} -> {predicted:12} {status} ({execution_time:.1f}s)")

    except Exception as e:
        print(f"處理範例 {i+1} 時發生錯誤: {e}")
        results.append({
            "uuid": uuid,
            "expected": expected,
            "predicted": "ERROR",
            "correct": False,
            "time": 0
        })

# 計算準確率
accuracy = correct / len(examples) if examples else 0
avg_time = total_time / len(examples) if examples else 0

print(f"\n📊 協作智能系統結果:")
print(f"準確率: {accuracy:.2%} ({correct}/{len(examples)})")
print(f"平均執行時間: {avg_time:.2f} 秒/例")
print(f"總執行時間: {total_time:.2f} 秒")

載入 1700 個訓練範例

在 12 個範例上評估協作智能系統...


協作智能處理:   8%|▊         | 1/12 [01:00<11:05, 60.52s/it]

範例  1: Contradiction -> Contradiction ✅ (60.5s)


協作智能處理:  17%|█▋        | 2/12 [02:15<11:30, 69.04s/it]

範例  2: Contradiction -> Contradiction ✅ (75.0s)


...

範例 10: Contradiction -> Contradiction ✅ (51.6s)


協作智能處理:  92%|█████████▏| 11/12 [10:53<00:57, 57.83s/it]

範例 11: Entailment   -> Entailment   ✅ (61.9s)


協作智能處理: 100%|██████████| 12/12 [11:52<00:00, 59.35s/it]

範例 12: Entailment   -> Contradiction ❌ (58.7s)

📊 協作智能系統結果:
準確率: 66.67% (8/12)
平均執行時間: 59.35 秒/例
總執行時間: 712.22 秒


## 產生提交檔案

使用我們的協作智能系統產生預測結果：

In [ ]:
def generate_collaborative_submission(test_file="test.json", output_file="collaborative_intelligence_submission.json", sample_size=None):
    """使用協作智能系統產生提交檔案"""

    # 載入測試資料
    try:
        with open(test_file, "r", encoding="utf-8") as f:
            test_data = json.load(f)
    except:
        print(f"❌ 無法載入測試資料 {test_file}")
        return

    examples = list(test_data.items())
    if sample_size:
        examples = examples[:sample_size]

    print(f"🚀 為 {len(examples)} 個範例產生協作智能預測...")

    submission = {}

    for i, (uuid, example) in enumerate(tqdm(examples, desc="協作智能處理")):
        try:
            statement = example.get("Statement")
            primary_id = example.get("Primary_id")
            secondary_id = example.get("Secondary_id")
            section_id = example.get("Section_id")

            if not statement or not primary_id:
                submission[uuid] = {"Prediction": "Contradiction"}
                continue

            # 獲取預測
            prediction = collaborative_intelligence_pipeline(
                statement=statement,
                primary_id=primary_id,
                secondary_id=secondary_id,
                section_id=section_id,
                verbose=False
            )

            submission[uuid] = {"Prediction": prediction}

        except Exception as e:
            print(f"處理 {uuid} 時發生錯誤: {e}")
            submission[uuid] = {"Prediction": "Contradiction"}

    # 儲存提交檔案
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(submission, f, indent=2)

    print(f"✅ 協作智能提交檔案已儲存至 {output_file}")
    return submission

# 產生小樣本提交
collaborative_submission = generate_collaborative_submission(
    test_file="test.json",
    output_file="collaborative_intelligence_submission.json",
    sample_size=10
)

print(f"為 {len(collaborative_submission)} 個範例產生了預測")

🚀 為 10 個範例產生協作智能預測...


協作智能處理: 100%|██████████| 10/10 [10:04<00:00, 60.49s/it]

✅ 協作智能提交檔案已儲存至 collaborative_intelligence_submission.json
為 10 個範例產生了預測


## 結論

### 協作智能系統優勢：
1. **團隊協作**: 模擬真實的專業團隊決策過程
2. **多重視角**: 從醫學、技術、品質等不同角度分析
3. **共識決策**: 通過協商達成平衡的判斷
4. **專業分工**: 每個角色都有明確的職責範圍
5. **品質保證**: 內建的檢查和驗證機制

### 四角色協作架構：
- **臨床研究主管**: 提供醫學專業指導和整體協調
- **資料科學家**: 進行技術驗證和統計分析
- **品質保證員**: 執行邏輯檢查和品質控制
- **決策委員會**: 整合所有意見並達成最終共識

### 協作流程特色：
- **順序協作**: 每個角色建立在前面的分析基礎上
- **交叉驗證**: 不同專業視角的相互印證
- **共識機制**: 最終決策反映團隊集體智慧
- **品質把關**: 多層次的檢查確保結果可靠性

### 適用場景：
- 需要多專業協作的複雜決策
- 要求高可靠性的臨床分析
- 團隊決策流程的模擬
- 需要透明化決策過程的應用

這個協作智能系統展示了如何通過模擬專業團隊的協作模式來提高臨床試驗NLP分析的品質和可靠性，每個角色都貢獻其專業優勢，最終達成團隊共識。